BaggageAI Computer Vision

Case Study:- Image Processing

IMPRTING REQUIRED LIBRARIES

In [70]:
import cv2
import imutils
from PIL import Image
import glob
import numpy as np

ROTATE BOUND FUNCTION for BETTER CROPPING of IMAGES

In [71]:
def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 0.57)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

LOAD THREAT IMAGES

In [72]:
image_list = [Image.open(item) for i in [glob.glob('D:\\Documents\\baggageAI\\threat_images\\*.%s' % ext) 
            for ext in ["jpg","gif","png","tga"]] for item in i]

MASKING

In [74]:
masked=[]
image_no = 1

#transparency percentage
TRANSPARENCY=60


for img in image_list:
  mask_2=img.convert("L")
  th=240
  mask_2 = mask_2.point(lambda i: i < th and 255) 
  paste_mask_2 = mask_2.split()[0].point(lambda i: i * TRANSPARENCY / 100.)
  masked.append(paste_mask_2)

  #Saving mask to a directory for use
  name = "D:\\Documents\\baggageAI\\masks\\" + 'mask' + str(image_no) + '.jpg'
  paste_mask_2.save(name, 'JPEG')
  image_no += 1

RESIZING, ROTATING THREAT IMAGE and MASKS

In [78]:

images_list=[]
for filename in glob.glob("D:\\Documents\\baggageAI\\background_images\\*.jpg"):
    im=Image.open(filename)
    images_list.append(im.size[1])

In [77]:
image_no=1

for (filename,i) in zip(glob.glob("D:\\Documents\\baggageAI\\threat_images\\*.jpg"),images_list):
    images=cv2.imread(filename)  
    image = imutils.resize(images,height=i) 
    rotate = rotate_bound(image, 45) #rotate by 45 degrees
    name = ("D:\\Documents\\baggageAI\\resized\\")+'resized' + str(image_no) + '.png'
    cv2.imwrite(name, rotate)
    image_no += 1

In [82]:
image_no=1

for (filename,i) in zip(masked,images_list):
    images = cv2.cvtColor(np.array(filename),cv2.COLOR_GRAY2BGR) #reading after converting to numpy array and saved in PIL format 
    image = imutils.resize(images,height=i)
    rotate = rotate_bound(image, 45)
    name = ("D:\\Documents\\baggageAI\\resized_masks\\")+'resized_mask' + str(image_no) + '.png'
    cv2.imwrite(name, rotate)
    image_no += 1

l = [1,2,3,4,5]

PASTING THREAT IMAGE on BACKGROUND IMAGE

In [83]:
image_no=1

for (filename,i,j) in zip(glob.glob("D:\\Documents\\baggageAI\\background_images\\*.jpg"),l,l):
  im1=Image.open(filename) 
  back_im = im1.copy()
  im2 = Image.open("D:\\Documents\\baggageAI\\resized\\"+'resized'+str(i)+'.png') #resized and rotated threat image
  mask_im = Image.open("D:\\Documents\\baggageAI\\resized_masks\\"+'resized_mask'+str(j)+'.png') #resized and rotated mask
  mask_imm=mask_im.convert('L')#covert to single channel image
  back_im.paste(im2, (48,40), mask_imm) 
  name = "D:\\Documents\\baggageAI\\final_out\\"+'output' + str(image_no) + '.png'
  back_im.save(name, quality=95)
  image_no += 1